컨볼루션 계산량 측정

In [1]:
def conv_output(k, s, p, W) :
    return ( W - k + 2*p) / s + 1

print(conv_output(7, 4, 3, 256))
print(conv_output(5, 1, 2, 256))
print(conv_output(3, 2, 1, 256))

64.75
256.0
128.5


In [3]:
tuple1 = (32, 64, 160, 320)
divisor = 4
result = tuple(map(lambda x: x // divisor, tuple1))
print(result)  # 출력: (5.0, 10.0, 15.0)

(8, 16, 40, 80)


모델 작동 테스트

In [5]:
from dataset import CarbonDataset, CarbonDataset_csv    
from models.carbonformer import CarbonFormer_v1

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from models.util import select_device
from models.metrics import CarbonLoss

device = select_device()

fp_forest = 'AP25_City_IMAGE.csv'
fp_city = 'AP25_Forest_IMAGE.csv'
args = {
        'dims': (32, 64, 160, 256),
        'heads': (1, 2, 5, 8),
        'ff_expansion': (8, 8, 4, 4),
        'reduction_ratio': (8, 4, 2, 1),
        'num_layers': 2,
        'decoder_dim': 256,
        'num_classes': 4,    
        'stage_kernel_stride_pad': [(7, 2, 3), 
                                   (3, 2, 1), 
                                   (3, 2, 1), 
                                   (3, 2, 1)],
        
}
model = CarbonFormer_v1(**args)
model.to(device)
loss = CarbonLoss(num_classes=4,cls_lambda=1,reg_lambda=0.0005).to(device)
# 데이터셋을 위한 변환 정의
image_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])
sh_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

label_transform = transforms.Compose([
    transforms.Resize((128, 128)), 
])
resizer = transforms.Compose([
    transforms.Resize((128, 128))
])
    

# 데이터셋 및 데이터 로더 생성
# forest_train_dataset = CarbonDataset(fp_forest, image_transform, sh_transform, label_transform,mode="Train")
# forest_train_loader = DataLoader(forest_train_dataset, batch_size=1, shuffle=True,pin_memory=True)

forest_val_dataset = CarbonDataset_csv(fp_forest, image_transform,sh_transform, label_transform,mode="Valid")
forest_val_loader = DataLoader(forest_val_dataset, batch_size=1, shuffle=True,pin_memory=True)

# city_train_dataset = CarbonDataset(fp_city, image_transform, sh_transform, label_transform,mode="Train")
# city_train_loader = DataLoader(city_train_dataset, batch_size=1, shuffle=True,pin_memory=True)

city_val_dataset = CarbonDataset_csv(fp_city, image_transform,sh_transform, label_transform,mode="Valid")
city_val_loader = DataLoader(city_val_dataset, batch_size=1, shuffle=True,pin_memory=True)


In [6]:
from thop import profile
from thop import clever_format
# Get a batch of data
batch = next(iter(city_val_loader))

# Extract the input from the batch
x = batch[0].to(device)
sh = batch[1].to(device)
carbon = batch[2].to(device)
gt = batch[3].to(device)

# Pass the input through the model
gt_pred,carbon_pred = model(x,sh)

# Print the output
print(gt_pred.shape)
print(carbon_pred.shape)

flops, params = profile(model, inputs=(x,sh))

# FLOPs를 사람이 읽기 쉬운 형식으로 변환
flops, params = clever_format([flops, params], "%.3f")

print(f"FLOPs: {flops}")
print(f"파라미터 수: {params}")


torch.Size([1, 4, 128, 128])
torch.Size([1, 1, 128, 128])
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_upsample() for <class 'torch.nn.modules.upsampling.Upsample'>.
FLOPs: 19.056G
파라미터 수: 8.520M
